In [1]:

# conda install -c conda-forge xesmf
# pip install cftime

import dask
# from dask_gateway import Gateway

import s3fs
import gcsfs
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
import zarr
import cftime
import tqdm
import datetime

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
plt.rcParams['figure.figsize'] = 12, 6

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
### data loading function
fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
gcs = gcsfs.GCSFileSystem(token='anon')

In [3]:
s3_url = 's3://cmip6-pds/cmip6-zarr-consolidated-stores.csv'
gg_url = 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv'

In [4]:
### read data index
df = pd.read_csv(gg_url)
df.tail(5)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
494017,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ta,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
494018,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,wap,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
494019,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,hur,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
494020,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,va,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
494021,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ua,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416


In [5]:
### find the model that has both historical and scenario records 
df_his_585 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_585 = df.query("variable_id == 'pr' & experiment_id == 'ssp585'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_585  = pd.merge(df_his_585, df_ssp_585, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_245 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_245 = df.query("variable_id == 'pr' & experiment_id == 'ssp245'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_245  = pd.merge(df_his_245, df_ssp_245, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_126 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_126 = df.query("variable_id == 'pr' & experiment_id == 'ssp126'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_126  = pd.merge(df_his_126, df_ssp_126, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

### get intersection
historical_url = list(set(df_merged_ssp_245['zstore_x']).intersection(set(df_merged_ssp_126['zstore_x'])).intersection(set(df_merged_ssp_585['zstore_x'])))

In [6]:
print('number of models: ', len(historical_url))
historical_url.sort()
historical_url[:5]

number of models:  24


['gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/pr/gn/v20181126/',
 'gs://cmip6/CMIP6/CMIP/CCCR-IITM/IITM-ESM/historical/r1i1p1f1/day/pr/gn/v20191226/',
 'gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/day/pr/gn/v20190429/',
 'gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/historical/r1i1p1f1/day/pr/gn/v20200616/',
 'gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/day/pr/gn/v20210114/']

### read and store historical data

In [7]:
# dictonary to collect data
ds_his_dictonary = {}
for i in tqdm.tqdm(range(len(historical_url))):
    zstore = historical_url[i]

   ### load data, use_cftime=True helps line up with the date format
    try:
        mapper = fs.get_mapper(zstore.replace("gs://cmip6", "s3://cmip6-pds"))
        ds_his = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)
    except KeyError:
        mapper = gcs.get_mapper(zstore.replace("s3://cmip6-pds", "gs://cmip6"))
        ds_his = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)

    ### keep data within 1950-2014
    ds_his = ds_his.sel(time = slice('1950', '2014'))
    min_year = ds_his.time.values.min().year
    max_year = ds_his.time.values.max().year
    list_of_year = range(min_year, max_year + 1)
    if min(list_of_year) != 1950:
        print(str(i) + ': min year of the model is not 1950! It is: ', str(min(list_of_year)))
    if max(list_of_year) != 2014:
        print(str(i) + ': max year of the model is not 2014! It is: ', str(max(list_of_year)))

    ### drop model with 360 days
    for y in list_of_year:
        days = len(ds_his.sel(time = str(y)).time.values)
        if days == 360:
            print('model ' + str(i) + ' has 360 days in a year!!')
            break
            
    else:
        ### Drop leap day
        ds_his = ds_his.where((ds_his['time.month'] != 2) | (ds_his['time.day'] != 29), drop=True)
        days = len(ds_his.sel(time = str(y)).time.values)
        if days != 365:
            print('model ' + str(i) + ' year ' + str(y) + ' does not have 365 days! It has ' + str(days) + ' days!!' )
        ds_his_dictonary[i] = ds_his

 92%|█████████▏| 22/24 [00:37<00:03,  1.57s/it]

model 21 has 360 days in a year!!


100%|██████████| 24/24 [00:40<00:00,  1.71s/it]


### Check if any model is missing lat/lon bounds

In [8]:
for i in tqdm.tqdm(ds_his_dictonary.keys()):
    try:
        ds_his_dictonary[i].lat_bnds
    except AttributeError:
        print('historical model ' + str(i) + ' does not have lat bounds!!')
    try:
        ds_his_dictonary[i].lon_bnds
    except AttributeError:
        print('historical model ' + str(i) + ' does not have lon bounds!!')

100%|██████████| 23/23 [00:00<00:00, 2772.73it/s]

historical model 12 does not have lat bounds!!
historical model 12 does not have lon bounds!!


### Regrid + cftime to numpy.dateframe

In [9]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.0, 1.0)),
                     'lat_bnds' : (['lat', 'bnds'], np.array([[x, x + 1] for x in range(-90,90)])),
                     'lon': (['lon'], np.arange(0.5, 360.0, 1.0)),
                     'lon_bnds' : (['lon', 'bnds'], np.array([[x, x + 1] for x in range(0,360)])),
                    })

for i in tqdm.tqdm(ds_his_dictonary.keys()):
    regridder = xe.Regridder(ds_his_dictonary[i], ds_out, 'nearest_s2d', reuse_weights=False)
    ds_his_dictonary[i] = regridder(ds_his_dictonary[i]) 
    regridder._grid_in = None
    regridder._grid_out = None
    ds_his_dictonary[i]['time'] = ds_his_dictonary[i].indexes['time'].to_datetimeindex().normalize()

print('Historical - Done!')

  0%|          | 0/23 [00:00<?, ?it/s]

Create weight file: nearest_s2d_160x320_180x360.nc


  4%|▍         | 1/23 [00:00<00:11,  1.84it/s]

using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.
Create weight file: nearest_s2d_94x192_180x360.nc


  9%|▊         | 2/23 [00:00<00:10,  2.10it/s]

...

using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.
Create weight file: nearest_s2d_180x288_180x360.nc


 96%|█████████▌| 22/23 [00:10<00:00,  2.20it/s]

using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.
Overwrite existing file: nearest_s2d_96x192_180x360.nc 
 You can set reuse_weights=True to save computing time.


100%|██████████| 23/23 [00:10<00:00,  2.18it/s]

using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.
Historical - Done!


### Mean & Std Calculation for historical

In [12]:
# create new dimension "model"
ds_his_temp = ds_his_dictonary[0].expand_dims({'model':list(ds_his_dictonary.keys())})
for i in tqdm.tqdm(ds_his_dictonary.keys()):
    ds_his_temp['pr'] = xr.where(ds_his_temp.model == i, ds_his_dictonary[i].pr, ds_his_temp.pr)
ds_his_temp

100%|██████████| 23/23 [00:00<00:00, 68.85it/s]


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, model: 23, time: 23725)
Coordinates:
  * model      (model) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 22 23
  * time       (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2014-12-31
    time_bnds  (time, bnds) object dask.array<chunksize=(23725, 1), meta=np.ndarray>
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    pr         (model, time, lat, lon) float64 dask.array<chunksize=(23, 46, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  nearest_s2d

In [1]:
ds_his_output = ds_his_temp.mean('model').rename_vars({'pr':'pr_mean'})
ds_his_output['std'] = ds_his_temp.std('model').pr
ds_his_output

NameError: name 'ds_his_temp' is not defined

In [1]:
ds_his_output.to_netcdf('historical_daily_pr.nc')

NameError: name 'ds_his_output' is not defined

In [1]:
#!aws s3 cp historical_daily.nc s3://aer-astd-mcclim/historical/historical_daily.nc

upload: ./historical_daily.nc to s3://aer-astd-mcclim/historical/historical_daily.nc


### Historical Monthly

In [13]:
ds_his_output['CDD'] = xr.where(((ds_his_output.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds_his_output.tas_mean - 273.15) * 9/5 + 32) - 65, 0)
ds_his_output['HDD'] = xr.where(((ds_his_output.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds_his_output.tas_mean - 273.15) * 9/5 + 32), 0)

ds_his_output_monthly = ds_his_output.resample(time = 'M').sum(dim = 'time')[['CDD', 'HDD']]
ds_his_output_monthly

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 780)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-31 1950-02-28 ... 2014-12-31
    height   float64 2.0
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Data variables:
    CDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    HDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>

In [ ]:
ds_his_output

### Output netcdf file

In [ ]:
ds_his_output_monthly.to_netcdf('historical_monthly.nc')

In [ ]:
!aws s3 cp historical_monthly.nc s3://aer-astd-mcclim/historical/historical_monthly.nc

### Verify the result

In [24]:
t = 0
n = 0
for i in range(0, 25):
    if i not in [22]:
        t += ds_his_dictonary[i].sel(time = '1988-04-22', lon = 215.5, lat = 21.5).tas.values
        n += 1
t/n

294.3251075744629

In [26]:
ds_daily = xr.open_dataset("historical_daily.nc")
ds_monthly = xr.open_dataset("historical_monthly.nc")

In [27]:
ds_daily.sel(time = '1988-04-22', lon = 215.5, lat = 21.5).tas_mean.values

array(294.32510757)

In [60]:
ds = ds_daily.sel(time = '1952-12', lon = 111.5, lat = 21.5)

In [61]:
ds['HDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds.tas_mean - 273.15) * 9/5 + 32), 0)
ds['CDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds.tas_mean - 273.15) * 9/5 + 32) - 65, 0)

In [62]:
print('CDD: ', ds.CDD.values.sum())
print('HDD: ', ds.HDD.values.sum())

CDD:  13.158066101074752
HDD:  41.9712652587883


In [63]:
d = ds_monthly.sel(time = '1952-12', lon = 111.5, lat = 21.5)
print('CDD: ', d.CDD.values)
print('HDD: ', d.HDD.values)

CDD:  [13.1580661]
HDD:  [41.97126526]
